<a href="https://colab.research.google.com/github/AndrewstheBuilder/VideoSemanticSearch_JiuJitsu/blob/main/CNN_JiuJitsu_Film.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Train and test on dataset

In [1]:
!unzip /content/trainset.zip -d /content/

Archive:  /content/trainset.zip
   creating: /content/trainset(2)/guarded/
  inflating: /content/trainset(2)/guarded/frame_10980.jpg  
  inflating: /content/trainset(2)/guarded/frame_11070.jpg  
  inflating: /content/trainset(2)/guarded/frame_11160.jpg  
  inflating: /content/trainset(2)/guarded/frame_11250.jpg  
  inflating: /content/trainset(2)/guarded/frame_11340.jpg  
  inflating: /content/trainset(2)/guarded/frame_11430.jpg  
  inflating: /content/trainset(2)/guarded/frame_11520.jpg  
  inflating: /content/trainset(2)/guarded/frame_11610.jpg  
  inflating: /content/trainset(2)/guarded/frame_11700.jpg  
  inflating: /content/trainset(2)/guarded/frame_11790.jpg  
  inflating: /content/trainset(2)/guarded/frame_16200.jpg  
  inflating: /content/trainset(2)/guarded/frame_16290.jpg  
  inflating: /content/trainset(2)/guarded/frame_16560.jpg  
  inflating: /content/trainset(2)/guarded/frame_16920.jpg  
  inflating: /content/trainset(2)/guarded/frame_17010.jpg  
  inflating: /content/tra

In [8]:
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [9]:
transform = transforms.Compose(
    [ transforms.Resize((64, 64)),
      transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_data_path = './trainset/'
train_dataset = datasets.ImageFolder(root=train_data_path, transform=transform)

FileNotFoundError: [Errno 2] No such file or directory: './trainset/'

In [ ]:
batch_size = 4 # You can adjust the batch size based on memory availability
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
classes = train_dataset.classes

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image
classes = train_dataset.classes

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(train_loader)
images, labels = next(dataiter)

# show images
imshow(torchvision.utils.make_grid(images))

# print labels
print(' '.join(f'{classes[labels[j]]:5s}' for j in range(batch_size)))

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding='same')
        self.bn1 = nn.BatchNorm2d(32)

        self.conv2 = nn.Conv2d(32, 64, 3, padding='same')
        self.bn2 = nn.BatchNorm2d(64)

        self.conv3 = nn.Conv2d(64, 128, 3, padding='same')
        self.bn3 = nn.BatchNorm2d(128)

        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(128 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 84)
        self.fc3 = nn.Linear(84, len(classes))

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
epochs=20

In [ ]:
net.train()
for epoch in range(epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        # for name, param in net.named_parameters():
        #   if param.requires_grad:
        #       print(f'{name} grad: {param.grad.abs().mean()}')
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % batch_size == batch_size-1:    # print every batch_size mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / batch_size:.3f}')
            running_loss = 0.0

print('Finished Training')

In [ ]:
print('labels',labels)
classes = train_dataset.classes
print('classes',classes)
t = torch.tensor([[[1, 2],[3, 4]],[[5, 6],[7, 8]]])
print('t.shape',t.shape)
t_flat = torch.flatten(t, 1)
print('t_flat.shape',t_flat.shape)

In [ ]:
PATH = './kaggle_net.pth'
torch.save(net.state_dict(), PATH)

In [ ]:
dataiter = iter(train_loader)
images, labels = next(dataiter)

# print images
imshow(torchvision.utils.make_grid(images))
print('GroundTruth: ', ' '.join(f'{classes[labels[j]]:5s}' for j in range(4)))

In [ ]:
net = Net()
net.load_state_dict(torch.load(PATH, weights_only=True))

In [ ]:
outputs = net(images)

In [ ]:
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join(f'{classes[predicted[j]]:5s}'
                              for j in range(4)))

In [ ]:

correct = 0
total = 0
# Set the model to evaluation mode
net.eval()

# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in train_loader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 training images: {100 * correct // total} %')

In [ ]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}

# Set the model to evaluation mode
net.eval()

# again no gradients needed
with torch.no_grad():
    for data in train_loader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1


# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

### Validate model on test set

In [1]:
!unzip /content/testset.zip -d /content/

Archive:  ./testset.zip
   creating: ./testset/dataset/
   creating: ./testset/dataset/guarded/
  inflating: ./testset/dataset/guarded/frame_5490.jpg  
  inflating: ./testset/dataset/guarded/frame_5310.jpg  
  inflating: ./testset/dataset/guarded/frame_6480.jpg  
  inflating: ./testset/dataset/guarded/frame_6840.jpg  
  inflating: ./testset/dataset/guarded/frame_3240.jpg  
  inflating: ./testset/dataset/guarded/frame_6930.jpg  
  inflating: ./testset/dataset/guarded/frame_4680.jpg  
  inflating: ./testset/dataset/guarded/frame_7290.jpg  
  inflating: ./testset/dataset/guarded/frame_4950.jpg  
  inflating: ./testset/dataset/guarded/frame_7200.jpg  
  inflating: ./testset/dataset/guarded/frame_8190.jpg  
  inflating: ./testset/dataset/guarded/frame_1890.jpg  
  inflating: ./testset/dataset/guarded/frame_4860.jpg  
   creating: ./testset/dataset/pinning/
  inflating: ./testset/dataset/pinning/frame_3420.jpg  
  inflating: ./testset/dataset/pinning/frame_5580.jpg  
  inflating: ./testset/d